<a href="https://colab.research.google.com/github/Edna-soso/DataAnalysis_Project/blob/main/Model/RFM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%matplotlib inline
from google.colab import files
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
from matplotlib.gridspec import GridSpec
import seaborn as sns
import plotly.express as px
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler as ss

In [126]:
data = pd.read_csv('/content/drive/MyDrive/python_OP/Clean_Dataset.csv')

In [127]:
class Data_to_RFM:
  def __init__(self,data):
    self.data=data
  def Datetime(self):
    self.data['InvoiceDate']=pd.to_datetime(self.data['InvoiceDate'])
  def To_RFM(self):
    Latest_date = dt.datetime(2011,12,10)
    RFM_data = self.data.groupby('CustomerID').agg({'InvoiceDate' : lambda x :(Latest_date - x.max()).days,
                                          'InvoiceNo' : 'count','TotalCost' : 'sum'}).reset_index()
    return RFM_data

In [128]:
df=Data_to_RFM(data)
df.Datetime()
df.To_RFM()
#RFM_data

,CustomerID,InvoiceDate,InvoiceNo,TotalCost
0,12346.0,325,1,77183.60
1,12347.0,2,182,4310.00
2,12348.0,75,31,1797.24
3,12349.0,18,73,1757.55
4,12350.0,310,17,334.40
...,...,...,...,...
4334,18280.0,277,10,180.60
4335,18281.0,180,7,80.82
4336,18282.0,7,12,178.05
4337,18283.0,3,756,2094.88


In [115]:
#RFM_data = pd.read_csv('/content/drive/MyDrive/python_OP/RFM_data.csv')

In [129]:
class RFM(Data_to_RFM):
  def __init__(self,RFM_data):
    self.RFM_data=RFM_data
  def Rename_Col(self):
    return RFM_data.rename(columns = {'InvoiceDate' : 'Recency',
                          'InvoiceNo' : "Frequency",
                          'TotalCost' : "Monetary"},inplace = True)
  def Div_Part(self):
    quantiles = RFM_data.drop('CustomerID',axis = 1).quantile(q = [0.25,0.5,0.75])
    return quantiles
  def R_score(var,p,d):
    if var <= d[p][0.25]:
       return 1
    elif var <= d[p][0.50]:
        return 2
    elif var <= d[p][0.75]:
       return 3
    else:
        return 4
  def FM_score(var,p,d):
      if var <= d[p][0.25]:
          return 4
      elif var <= d[p][0.50]:
          return 3
      elif var <= d[p][0.75]:
          return 2
      else:
          return 1
  def Scoring(self):
    RFM_data['R_score'] = RFM_data['Recency'].apply(R_score,args = ('Recency',quantiles,))
    RFM_data['F_score'] = RFM_data['Frequency'].apply(FM_score,args = ('Frequency',quantiles,))
    RFM_data['M_score'] = RFM_data['Monetary'].apply(FM_score,args = ('TotalCost',quantiles,))
    RFM_data['RFM_Group'] = RFM_data['R_score'].astype(str) + RFM_data['F_score'].astype(str) + RFM_data['M_score'].astype(str)
    RFM_data['RFM_Score'] = RFM_data[['R_score','F_score','M_score']].sum(axis = 1)
  def Cus_Seg(self):
    loyalty_level = ['True Lover','Flirting','Potential lover','Platonic Friend']
    cuts = pd.qcut(RFM_data['RFM_Score'],q = 4,labels=loyalty_level)
    RFM_data['RFM_Loyality_level'] = cuts.values

In [132]:
df=RFM(RFM_data)
df.Rename_Col()
df.Div_Part()
df.R_score()
df.FM_score()
df.Scoring()
df.Cus_Seg()
#df.Scoring()

TypeError: ignored

In [131]:
RFM_data

,Unnamed: 0,CustomerID,Recency,Frequency,Monetary,R_score,F_score,M_score,RFM_Group,RFM_Score,RFM_Loyality_level
0,0,12347.0,2,182,4310.00,1,1,1,111,3,True Lover
1,1,12348.0,75,31,1797.24,3,3,1,331,7,Flirting
2,2,12349.0,18,73,1757.55,2,2,1,221,5,True Lover
3,3,12350.0,310,17,334.40,4,4,3,443,11,Platonic Friend
4,4,12352.0,36,85,2506.04,2,2,1,221,5,True Lover
...,...,...,...,...,...,...,...,...,...,...,...
4234,4234,18280.0,277,10,180.60,4,4,4,444,12,Platonic Friend
4235,4235,18281.0,180,7,80.82,4,4,4,444,12,Platonic Friend
4236,4236,18282.0,7,12,178.05,1,4,4,144,9,Potential lover
4237,4237,18283.0,3,754,2090.43,1,1,1,111,3,True Lover


In [ ]:
RFM_data.to_csv('RFM_data.csv') 
files.download('RFM_data.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>